In [ ]:
from dotenv import load_dotenv

load_dotenv()

from langchain_teddynote import logging

logging.langsmith("FinalProject")

In [ ]:
import os
import json
import glob

# 데이터가 저장된 폴더 이름
DATA_DIR = "../data"


def load_all_violation_data(directory):
    """
    지정된 디렉토리에서 'violation_n.json' 형식의 모든 파일을 찾아
    하나의 리스트로 합쳐서 반환합니다.
    """

    # data/violation_*.json 패턴에 맞는 모든 파일 경로를 찾습니다.
    file_pattern = os.path.join(directory, "violation_*.json")
    json_file_paths = glob.glob(file_pattern)

    # 파일 이름 순서대로 정렬 (예: violation_1, violation_2, ..., violation_10 순)
    # glob은 순서를 보장하지 않으므로, 수동 정렬이 필요할 수 있습니다.
    # 파일 이름에서 숫자 부분을 추출하여 정렬합니다.
    json_file_paths.sort(
        key=lambda x: int(os.path.basename(x).split("_")[1].split(".")[0])
    )

    if not json_file_paths:
        print(
            f"경고: '{directory}' 폴더에서 'violation_n.json' 파일을 찾을 수 없습니다."
        )
        return []

    # 모든 JSON 파일의 데이터를 담을 빈 리스트
    combined_data = []

    print(f"총 {len(json_file_paths)}개의 파일을 불러옵니다...")

    # 각 파일을 순회하며 데이터를 불러오고 리스트에 추가합니다.
    for file_path in json_file_paths:
        try:
            with open(file_path, "r", encoding="utf-8") as f:
                data_in_file = json.load(f)
                # extend를 사용하여 리스트 안의 모든 항목을 개별적으로 추가
                combined_data.extend(data_in_file)
        except json.JSONDecodeError:
            print(f"오류: '{file_path}' 파일이 올바른 JSON 형식이 아닙니다.")
        except Exception as e:
            print(f"'{file_path}' 파일을 읽는 중 오류가 발생했습니다: {e}")

    return combined_data

In [26]:
import os
from dotenv import load_dotenv
from langchain.schema.document import Document
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

# --- 사전 준비 (시뮬레이션을 위한 더미 함수 및 데이터) ---
DATA_DIR = "../data"

# --- OpenAI API 키 설정 ---
load_dotenv()
if "OPENAI_API_KEY" not in os.environ:
    print("OpenAI API 키가 설정되지 않았습니다.")
    exit()

# --- 벡터 DB를 저장할 경로 설정 ---
CHROMA_PATH_COSINE = "../chroma_db_cos"  # 코사인 유사도 DB를 위한 새 경로

# --- 데이터 로드 및 Document 객체 변환 (이전과 동일) ---
all_data = load_all_violation_data(DATA_DIR)
if not all_data:
    print("로드할 데이터가 없습니다.")
    exit()
print(f"\n총 {len(all_data)}개의 데이터 항목 로드 완료.")

documents = []
for item in all_data:
    if "증강데이터" in item:
        page_content = item.pop("증강데이터")
        metadata = item
        documents.append(Document(page_content=page_content, metadata=metadata))
print(f"'{len(documents)}'개의 LangChain Document 객체를 생성했습니다.")


# ==============================================================================
# ▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼
# 벡터 DB 생성 (코사인 유사도 설정 추가)
# ==============================================================================

# 1. OpenAI 임베딩 모델 초기화
print("\nOpenAI 임베딩 모델을 초기화합니다...")
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# 2. Chroma DB 생성 시 코사인 유사도(cosine) 설정 추가
print(
    f"코사인 유사도를 사용하여 Chroma DB 생성을 시작합니다. 저장 위치: '{CHROMA_PATH_COSINE}'"
)
db_cosine = Chroma.from_documents(
    documents=documents,
    embedding=embeddings,
    persist_directory=CHROMA_PATH_COSINE,
    # ⭐ 이 부분이 코사인 유사도를 설정하는 핵심입니다.
    collection_metadata={"hnsw:space": "cosine"},
)

print(
    f"\n✅ 총 {db_cosine._collection.count()}개의 문서가 성공적으로 임베딩되어 Chroma DB에 저장되었습니다."
)
# ==============================================================================
# ▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲
# ==============================================================================

총 22개의 파일을 불러옵니다...

총 220개의 데이터 항목 로드 완료.
'220'개의 LangChain Document 객체를 생성했습니다.

OpenAI 임베딩 모델을 초기화합니다...


Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


코사인 유사도를 사용하여 Chroma DB 생성을 시작합니다. 저장 위치: '../chroma_db_cos'

✅ 총 220개의 문서가 성공적으로 임베딩되어 Chroma DB에 저장되었습니다.


In [28]:
# --- (권장) 저장된 DB 로드 및 '유사도 점수'와 함께 검색 테스트 ---
print("\n--- 저장된 코사인 유사도 DB 로드 및 검색 테스트 ---")

# DB 로드
db_loaded_cosine = Chroma(
    persist_directory=CHROMA_PATH_COSINE, embedding_function=embeddings
)

# 쿼리 예시
q1 = '#{발신 스페이스} 요금 미납으로 서비스 사용 정지되었습니다.  미납 전액 납부하셔야 정지 해제되므로 빠른 납부 바랍니다. ▶미납금액: #{미납금액}  자세한 내용은 아래 "자세히보기" 버튼을 클릭해 주세요.'

query = """ 
#{발신 스페이스} 요금 미납으로 서비스 사용 정지되었습니다.  미납 전액 납부하셔야 정지 해제되므로 빠른 납부 바랍니다. ▶미납금액: #{미납금액}  자세한 내용은 아래 "자세히보기" 버튼을 클릭해 주세요.
"""
search_results_with_score = db_loaded_cosine.similarity_search_with_score(query, k=2)

print(f"\n[쿼리]: {query}")
print("\n[검색 결과 (코사인 유사도 점수 포함)]:")
for doc, score in search_results_with_score:
    print(f"  - 유사도 점수: {score:.4f}")  # 소수점 4자리까지 출력
    print(f"    - 내용: {doc.page_content}")
    print(f"    - 출처 법규: {doc.metadata.get('규정', 'N/A')}")
    print(f"    - 출처 법규: {doc.metadata.get('위반상세', 'N/A')}")
    print("-" * 20)

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given



--- 저장된 코사인 유사도 DB 로드 및 검색 테스트 ---

[쿼리]:  
#{발신 스페이스} 요금 미납으로 서비스 사용 정지되었습니다.  미납 전액 납부하셔야 정지 해제되므로 빠른 납부 바랍니다. ▶미납금액: #{미납금액}  자세한 내용은 아래 "자세히보기" 버튼을 클릭해 주세요.


[검색 결과 (코사인 유사도 점수 포함)]:
  - 유사도 점수: 0.3806
    - 내용: [미납안내] #{고객명}님, 통신요금이 미납되었습니다. 아래 버튼을 눌러 바로 결제하세요. [버튼: 지금 바로 납부하기]
    - 출처 법규: R-04
    - 출처 법규: (10) 결제/송금/납부 유도
--------------------
  - 유사도 점수: 0.4145
    - 내용: 회비 납부 마감일입니다. 원활한 서비스 이용을 위해 빠른 납부 부탁드립니다. [버튼: 회비 납부]
    - 출처 법규: R-04
    - 출처 법규: (10) 결제/송금/납부 유도
--------------------
